In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib

plt.style.use('ggplot')

## Import datasets

In [433]:
diabetes = pd.read_csv('dataset_diabetes/diabetic_data.csv')

## Preview of two datasets

In [307]:
pd.options.display.max_columns = 55

In [308]:
diabetes.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [309]:
print(diabetes.shape)

(101766, 50)


### Split id_mapping into 3 unique dataframes

In [310]:
f = open('dataset_diabetes/IDs_mapping.csv', 'r')    # 'r' for read
lines = f.readlines()
f.close()

In [311]:
indices = [i for i, x in enumerate(lines) if x == ',\n']

In [312]:
indices

[9, 41]

In [383]:
admission_type = pd.read_csv('dataset_diabetes/IDs_mapping.csv', skiprows=list(range(9,len(lines))))

rows_skip = list(range(0,10)) + list(range(41, len(lines)))
discharge_disposition = pd.read_csv('dataset_diabetes/IDs_mapping.csv', skiprows=rows_skip)

admission_source = pd.read_csv('dataset_diabetes/IDs_mapping.csv', skiprows=list(range(42)))

### Combine values in all three ID tables

admission_type

In [384]:
## make 'Not Available', 'NaN', and 'Not Mapped' all 'NA'
for i in range(len(admission_type)):
    x = admission_type['description'][i]
    if (x == 'Not Available' or x == 'Not Mapped'):
        admission_type['description'][i] = np.nan

/home/anisha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


discharge_disposition

In [386]:
## make 'Unknown/Invalid', 'NaN', and 'Not Mapped' all 'NA'
for i in range(len(discharge_disposition)):
    x = discharge_disposition['description'][i]
    if (x == 'Unknown/Invalid' or x == 'Not Mapped'):
        discharge_disposition['description'][i] = np.nan
    elif(pd.isna(x)):
        continue
    elif ('Expired' in x):
        discharge_disposition['description'][i] = 'Expired'

/home/anisha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/anisha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


admission_source

In [394]:
## make 'Not Available', 'NaN', 'Unknown/Invalid' and 'Not Mapped' all 'NA'
for i in range(len(admission_source)):
    x = admission_source['description'][i]
    if(pd.isna(x)):
        continue
    elif (x.strip() == 'Not Available' or x.strip() == 'Not Mapped' or x.strip() == 'Unknown/Invalid'):
        admission_source['description'][i] = np.nan

/home/anisha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


## Merge Datasets

In [434]:
ds = diabetes.merge(admission_type, how = 'left', on='admission_type_id')
ds2 = ds.merge(discharge_disposition, how = 'left', on = 'discharge_disposition_id')
full_dataset = ds2.merge(admission_source, how = 'left', on = 'admission_source_id')
full_dataset.rename(columns={"description_x": "admissionType",
                             "description_y": "dischargeDisposition",
                             "description" : "admissionSource"}, inplace = True)

In [398]:
full_dataset.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admissionType,dischargeDisposition,admissionSource
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,NaN,NaN,Physician Referral
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30,Emergency,Discharged to home,Emergency Room
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO,Emergency,Discharged to home,Emergency Room
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO,Emergency,Discharged to home,Emergency Room
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,Emergency,Discharged to home,Emergency Room


#### Remove admission_type_id, discharge_disposition_id, and admission_source_id

In [435]:
full_dataset.drop(['admission_type_id', 'discharge_disposition_id', 'admission_source_id'], axis = 1, inplace = True)

In [437]:
full_dataset.shape

(101766, 51)

### Make Response Variable - readmitted_30
#### Make readmitted_30 column
Keep column 'readmitted' because >30 could be an indicator

In [401]:
np.unique(full_dataset.readmitted.values)

array(['<30', '>30', 'NO'], dtype=object)

In [436]:
full_dataset['readmitted_30'] = ['Yes' if x == '<30' else 'No' for x in full_dataset['readmitted']]

### Remove unnecessary columns

#### Remove weight from dataset (97% missing)

In [438]:
full_dataset.drop(['weight'], axis = 1, inplace = True)

#### If only one value in column, remove column
This includes only citoglipton and examide
#### If less than 1000 (1%) not 'No' in column, remove column
Can change threshold number, calculated columns below

In [439]:
nos = []
for i in range(18,43):
    col = full_dataset.columns[i]
    numNotNo = len(full_dataset[full_dataset[col] != 'No'])
    if (numNotNo < 1000):
        nos.append(col)

In [440]:
nos

['nateglinide',
 'chlorpropamide',
 'acetohexamide',
 'tolbutamide',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone']

In [441]:
full_dataset.drop(nos, axis = 1, inplace = True)

In [442]:
full_dataset.shape

(101766, 35)

### Make columns for diagnosis categories
#### Convert icd9 codes to to categories
This is under the diag_1,2,3 column

In [443]:
def code_to_category(icd9code):
    try:
        icd9code = float(icd9code)
        if (icd9code >= 390 and icd9code < 460) or icd9code == 785:
            return 'Circulatory'
        if (icd9code >= 460 and icd9code < 520) or icd9code == 786:
            return 'Respiratory'
        if (icd9code >= 520 and icd9code < 580) or icd9code == 787:
            return 'Digestive'
        if (icd9code >= 250 and icd9code < 251):
            return 'Diabetes'
        if (icd9code >= 800 and icd9code < 1000):
            return 'Injury'
        if (icd9code >= 710 and icd9code < 740):
            return 'Musculoskeletal'
        if (icd9code >= 580 and icd9code < 630) or icd9code == 788:
            return 'Genitourinary'
        if (icd9code >= 140 and icd9code < 240) or icd9code == 780 or icd9code == 781 or icd9code == 784 or (icd9code >= 790 and icd9code < 800):
            return 'Neoplasms'
        if (icd9code >= 240 and icd9code < 250) or (icd9code >= 251 and icd9code < 280) or (icd9code >= 680 and icd9code < 710) or icd9code == 782:
            return 'Neoplasms'
        else:
            return 'Other'
    except:
        if (icd9code == '?'):
            return '?'
        return 'Other'

In [444]:
full_dataset['diag_1_cat'] = [code_to_category(x) for x in full_dataset['diag_1']]
full_dataset['diag_2_cat'] = [code_to_category(x) for x in full_dataset['diag_2']]
full_dataset['diag_3_cat'] = [code_to_category(x) for x in full_dataset['diag_3']]

In [445]:
full_dataset.drop(['diag_1', 'diag_2','diag_3'], axis = 1, inplace = True)

### Remove patients if have 'expired'
This remove 1,652 rows

In [446]:
expired = full_dataset[full_dataset['dischargeDisposition'] == 'Expired'].index
full_dataset.drop(expired, axis = 0, inplace = True)

In [447]:
full_dataset.shape

(100114, 35)

### Make only 3 values for medicines: No, Steady, Change
If column has 'Up' or 'Down', then change these observations to 'Change'.
Remove 'change' column, as this is indicated in medicine's columns.

In [468]:
for i in range(17,25):
    col = full_dataset.columns[i]
    full_dataset[col] = ['Change' if (x == 'Down' or x == 'Up') else x for x in full_dataset[col]]

What do we do about 'max_glu_serum' and 'A1Cresult'?

### Remove 'change' column.

In [470]:
full_dataset.drop('change', axis = 1, inplace = True)

In [471]:
full_dataset

,encounter_id,patient_nbr,race,gender,age,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,diabetesMed,readmitted,admissionType,dischargeDisposition,admissionSource,readmitted_30,diag_1_cat,diag_2_cat,diag_3_cat
0,2278392,8222157,Caucasian,Female,[0-10),1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,1,None,None,No,No,No,No,No,No,No,No,No,NO,NaN,NaN,Physician Referral,No,Diabetes,?,?
1,149190,55629189,Caucasian,Female,[10-20),3,?,?,59,0,18,0,0,0,9,None,None,No,No,No,No,No,No,No,Change,Yes,>30,Emergency,Discharged to home,Emergency Room,No,Neoplasms,Diabetes,Neoplasms
2,64410,86047875,AfricanAmerican,Female,[20-30),2,?,?,11,5,13,2,0,1,6,None,None,No,No,No,Steady,No,No,No,No,Yes,NO,Emergency,Discharged to home,Emergency Room,No,Other,Diabetes,Other
3,500364,82442376,Caucasian,Male,[30-40),2,?,?,44,1,16,0,0,0,7,None,None,No,No,No,No,No,No,No,Change,Yes,NO,Emergency,Discharged to home,Emergency Room,No,Other,Diabetes,Circulatory
4,16680,42519267,Caucasian,Male,[40-50),1,?,?,51,0,8,0,0,0,5,None,None,No,No,No,Steady,No,No,No,Steady,Yes,NO,Emergency,Discharged to home,Emergency Room,No,Neoplasms,Neoplasms,Diabetes
5,35754,82637451,Caucasian,Male,[50-60),3,?,?,31,6,16,0,0,0,9,None,None,No,No,No,No,No,No,No,Steady,Yes,>30,Urgent,Discharged to home,Clinic Referral,No,Circulatory,Circulatory,Diabetes
6,55842,84259809,Caucasian,Male,[60-70),4,?,?,70,1,21,0,0,0,7,None,None,Steady,No,Steady,No,No,No,No,Steady,Yes,NO,Elective,Discharged to home,Clinic Referral,No,Circulatory,Circulatory,Other
7,63768,114882984,Caucasian,Male,[70-80),5,?,?,73,0,12,0,0,0,8,None,None,No,No,No,No,Steady,No,No,No,Yes,>30,Emergency,Discharged to home,Emergency Room,No,Circulatory,Respiratory,Diabetes
8,12522,48330783,Caucasian,Female,[80-90),13,?,?,68,2,28,0,0,0,8,None,None,No,No,No,Steady,No,No,No,Steady,Yes,NO,Urgent,Discharged to home,Transfer from a hospital,No,Circulatory,Circulatory,Other
9,15738,63555939,Caucasian,Female,[90-100),12,?,InternalMedicine,33,3,18,0,0,0,8,None,None,No,No,No,No,No,No,Steady,Steady,Yes,NO,Elective,Discharged/transferred to SNF,Transfer from a hospital,No,Circulatory,Neoplasms,Respiratory


#### Impute Missing Values
Values missing from:
- race
- payer_code
- medical_speciality
- diag_1_cat
- diag_2_cat
- diag_3_cat